<a href="https://colab.research.google.com/github/2303A51531/23CSBTB39-40/blob/main/hpc_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import random
import cProfile
import pstats
import io
import tracemalloc

# -----------------------------
# Vector operations (pure Python)
# -----------------------------

def vector_add(a, b):
    """Element-wise vector addition using a Python loop."""
    n = len(a)
    result = [0.0] * n
    for i in range(n):
        result[i] = a[i] + b[i]
    return result


def dot_product(a, b):
    """Dot product using a Python loop."""
    total = 0.0
    for i in range(len(a)):
        total += a[i] * b[i]
    return total


# -----------------------------
# Benchmark + profiling wrapper
# -----------------------------

def run_benchmark(n):
    # Generate random vectors
    a = [random.random() for _ in range(n)]
    b = [random.random() for _ in range(n)]

    # ---- Timing ----
    start = time.perf_counter()
    c = vector_add(a, b)
    add_time = time.perf_counter() - start

    start = time.perf_counter()
    dp = dot_product(a, b)
    dot_time = time.perf_counter() - start

    return add_time, dot_time, dp


# -----------------------------
# Main execution
# -----------------------------

if __name__ == "__main__":
    VECTOR_SIZE = 1_000_000

    print(f"Vector size: {VECTOR_SIZE}")

    # ---- Memory tracking ----
    tracemalloc.start()

    # ---- Profiling ----
    profiler = cProfile.Profile()
    profiler.enable()

    add_time, dot_time, dp = run_benchmark(VECTOR_SIZE)

    profiler.disable()

    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    # ---- Results ----
    print("\nTiming Results (serial, pure Python loops)")
    print(f"Vector addition time: {add_time:.6f} s")
    print(f"Dot product time:     {dot_time:.6f} s")
    print(f"Dot product value:    {dp:.6f}")

    print("\nMemory Usage")
    print(f"Current memory: {current / 1e6:.2f} MB")
    print(f"Peak memory:    {peak / 1e6:.2f} MB")

    # ---- Profiling output ----
    s = io.StringIO()
    stats = pstats.Stats(profiler, stream=s)
    stats.strip_dirs().sort_stats("cumulative").print_stats(10)

    print("\nTop 10 functions by cumulative time:")
    print(s.getvalue())


Vector size: 1000000

Timing Results (serial, pure Python loops)
Vector addition time: 0.997107 s
Dot product time:     0.642218 s
Dot product value:    249807.492152

Memory Usage
Current memory: 0.05 MB
Peak memory:    96.94 MB

Top 10 functions by cumulative time:
         2000831 function calls (2000822 primitive calls) in 5.458 seconds

   Ordered by: cumulative time
   List reduced from 150 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      6/5    0.000    0.000    5.450    1.090 base_events.py:1922(_run_once)
        5    0.337    0.067    5.439    1.088 selectors.py:451(select)
        5    2.884    0.577    4.141    0.828 {built-in method time.sleep}
        1    0.228    0.228    2.042    2.042 ipython-input-984709179.py:33(run_benchmark)
  2000000    1.526    0.000    1.526    0.000 {method 'random' of '_random.Random' objects}
        1    0.394    0.394    0.394    0.394 ipython-input-984709179.py:12(vector_add)
   

In [2]:
import time
import random
import cProfile
import pstats
import io
import tracemalloc

# -----------------------------
# Matrix utilities
# -----------------------------

def generate_matrix(n):
    """Generate an n x n matrix filled with random floats."""
    return [[random.random() for _ in range(n)] for _ in range(n)]


def matmul_naive(A, B):
    """Naïve O(n^3) matrix multiplication using triple nested loops."""
    n = len(A)
    C = [[0.0] * n for _ in range(n)]

    for i in range(n):
        for j in range(n):
            s = 0.0
            for k in range(n):
                s += A[i][k] * B[k][j]
            C[i][j] = s

    return C


# -----------------------------
# Benchmark + profiling wrapper
# -----------------------------

def run_benchmark(n):
    A = generate_matrix(n)
    B = generate_matrix(n)

    start = time.perf_counter()
    C = matmul_naive(A, B)
    elapsed = time.perf_counter() - start

    # Touch result to avoid dead-code elimination concerns
    checksum = sum(C[0])

    return elapsed, checksum


# -----------------------------
# Main execution
# -----------------------------

if __name__ == "__main__":
    MATRIX_SIZE = 200  # Adjust carefully (O(n^3)!)

    print(f"Matrix size: {MATRIX_SIZE} x {MATRIX_SIZE}")

    tracemalloc.start()

    profiler = cProfile.Profile()
    profiler.enable()

    elapsed, checksum = run_benchmark(MATRIX_SIZE)

    profiler.disable()

    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    # ---- Results ----
    print("\nTiming Results (serial, naïve triple-loop)")
    print(f"Elapsed time: {elapsed:.6f} s")
    print(f"Checksum:     {checksum:.6f}")

    print("\nMemory Usage")
    print(f"Current memory: {current / 1e6:.2f} MB")
    print(f"Peak memory:    {peak / 1e6:.2f} MB")

    # ---- Profiling output ----
    s = io.StringIO()
    stats = pstats.Stats(profiler, stream=s)
    stats.strip_dirs().sort_stats("cumulative").print_stats(10)

    print("\nTop 10 functions by cumulative time:")
    print(s.getvalue())


Matrix size: 200 x 200

Timing Results (serial, naïve triple-loop)
Elapsed time: 2.691979 s
Checksum:     10233.720717

Memory Usage
Current memory: 0.05 MB
Peak memory:    3.92 MB

Top 10 functions by cumulative time:
         81079 function calls (81066 primitive calls) in 2.760 seconds

   Ordered by: cumulative time
   List reduced from 174 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      8/7    0.000    0.000    2.758    0.394 base_events.py:1922(_run_once)
        7    0.015    0.002    2.740    0.391 selectors.py:451(select)
        1    0.555    0.555    2.511    2.511 ipython-input-2721158177.py:36(run_benchmark)
        1    1.005    1.005    1.005    1.005 {built-in method time.sleep}
        1    0.950    0.950    0.950    0.950 ipython-input-2721158177.py:17(matmul_naive)
    80000    0.043    0.000    0.043    0.000 {method 'random' of '_random.Random' objects}
        2    0.007    0.003    0.032    0.016 ipytho

In [4]:
import time
import cProfile
import pstats
import io
import tracemalloc

# -----------------------------
# Image utilities
# -----------------------------

def generate_image(height, width):
    """Generate a synthetic 2D image (grayscale)."""
    return [[(i * j) % 256 for j in range(width)] for i in range(height)]


def blur_5x5(image):
    """
    Apply a 5x5 box blur using a naïve stencil.
    Border pixels are left unchanged.
    """
    h = len(image)
    w = len(image[0])

    output = [[0] * w for _ in range(h)]

    # Copy borders directly
    for i in range(h):
        output[i][0] = image[i][0]
        output[i][1] = image[i][1]
        output[i][-1] = image[i][-1]
        output[i][-2] = image[i][-2]

    for j in range(w):
        output[0][j] = image[0][j]
        output[1][j] = image[1][j]
        output[-1][j] = image[-1][j]
        output[-2][j] = image[-2][j]

    # 5x5 stencil (interior only)
    for i in range(2, h - 2):
        for j in range(2, w - 2):
            s = 0
            for di in range(-2, 3):
                for dj in range(-2, 3):
                    s += image[i + di][j + dj]
            output[i][j] = s // 25

    return output


# -----------------------------
# Benchmark + profiling wrapper
# -----------------------------

def run_benchmark(height, width):
    img = generate_image(height, width)

    start = time.perf_counter()
    out = blur_5x5(img)
    elapsed = time.perf_counter() - start

    # Touch output to prevent dead-code elimination
    checksum = sum(out[height // 2])

    return elapsed, checksum


# -----------------------------
# Main execution
# -----------------------------

if __name__ == "__main__":
    HEIGHT = 512
    WIDTH = 512

    print(f"Image size: {HEIGHT} x {WIDTH}")

    tracemalloc.start()

    profiler = cProfile.Profile()
    profiler.enable()

    elapsed, checksum = run_benchmark(HEIGHT, WIDTH)

    profiler.disable()

    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    # ---- Results ----
    print("\nTiming Results (serial, 5x5 stencil)")
    print(f"Elapsed time: {elapsed:.6f} s")
    print(f"Checksum:     {checksum}")

    print("\nMemory Usage")
    print(f"Current memory: {current / 1e6:.2f} MB")
    print(f"Peak memory:    {peak / 1e6:.2f} MB")

    # ---- Profiling output ----
    s = io.StringIO()
    stats = pstats.Stats(profiler, stream=s)
    stats.strip_dirs().sort_stats("cumulative").print_stats(10)

    print("\nTop 10 functions by cumulative time:")
    print(s.getvalue())


Image size: 512 x 512

Timing Results (serial, 5x5 stencil)
Elapsed time: 23.271091 s
Checksum:     51590

Memory Usage
Current memory: 0.04 MB
Peak memory:    4.33 MB

Top 10 functions by cumulative time:
         957 function calls (948 primitive calls) in 23.461 seconds

   Ordered by: cumulative time
   List reduced from 134 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      6/5    0.002    0.000   23.444    4.689 base_events.py:1922(_run_once)
        1    0.000    0.000   23.225   23.225 ipython-input-3790407121.py:55(run_benchmark)
       23   22.952    0.998   22.952    0.998 {built-in method time.sleep}
        1    0.273    0.273    0.273    0.273 ipython-input-3790407121.py:16(blur_5x5)
        4    0.039    0.010    0.206    0.051 selectors.py:451(select)
        1    0.133    0.133    0.133    0.133 ipython-input-3790407121.py:11(generate_image)
        4    0.034    0.009    0.034    0.009 {method 'poll' of 'select

In [5]:
import time
import random
import cProfile
import pstats
import io
import tracemalloc

# -----------------------------
# Monte Carlo Pi estimation
# -----------------------------

def estimate_pi(num_samples):
    """
    Estimate pi using Monte Carlo sampling.
    Random points are generated in the unit square.
    """
    inside = 0

    for _ in range(num_samples):
        x = random.random()
        y = random.random()
        if x * x + y * y <= 1.0:
            inside += 1

    return 4.0 * inside / num_samples


# -----------------------------
# Benchmark + profiling wrapper
# -----------------------------

def run_benchmark(num_samples):
    start = time.perf_counter()
    pi_est = estimate_pi(num_samples)
    elapsed = time.perf_counter() - start

    return elapsed, pi_est


# -----------------------------
# Main execution
# -----------------------------

if __name__ == "__main__":
    NUM_SAMPLES = 20_000_000  # Adjust carefully

    print(f"Number of samples: {NUM_SAMPLES}")

    tracemalloc.start()

    profiler = cProfile.Profile()
    profiler.enable()

    elapsed, pi_est = run_benchmark(NUM_SAMPLES)

    profiler.disable()

    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    # ---- Results ----
    print("\nTiming Results (serial, Monte Carlo π)")
    print(f"Elapsed time: {elapsed:.6f} s")
    print(f"Estimated π:  {pi_est:.6f}")

    print("\nMemory Usage")
    print(f"Current memory: {current / 1e6:.2f} MB")
    print(f"Peak memory:    {peak / 1e6:.2f} MB")

    # ---- Profiling output ----
    s = io.StringIO()
    stats = pstats.Stats(profiler, stream=s)
    stats.strip_dirs().sort_stats("cumulative").print_stats(10)

    print("\nTop 10 functions by cumulative time:")
    print(s.getvalue())


Number of samples: 20000000

Timing Results (serial, Monte Carlo π)
Elapsed time: 30.547712 s
Estimated π:  3.141922

Memory Usage
Current memory: 0.04 MB
Peak memory:    0.04 MB

Top 10 functions by cumulative time:
         40001132 function calls (40001108 primitive calls) in 30.549 seconds

   Ordered by: cumulative time
   List reduced from 172 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      8/7    0.000    0.000   30.546    4.364 base_events.py:1922(_run_once)
      2/1    0.002    0.001   30.311   30.311 ipython-input-667225315.py:32(run_benchmark)
       30   25.333    0.844   30.140    1.005 {built-in method time.sleep}
 40000000    4.871    0.000    4.871    0.000 {method 'random' of '_random.Random' objects}
        1    0.143    0.143    0.170    0.170 ipython-input-667225315.py:12(estimate_pi)
      8/5    0.000    0.000    0.029    0.006 events.py:86(_run)
      8/5    0.000    0.000    0.029    0.006 {method 'r

In [6]:
import time
import random
import cProfile
import pstats
import io
import tracemalloc

# -----------------------------
# Particle utilities
# -----------------------------

def generate_particles(n):
    """Generate n particles with random 2D positions."""
    return [(random.random(), random.random()) for _ in range(n)]


def pairwise_interactions(particles):
    """
    Compute a simplified pairwise interaction score.
    No physics integration; just interaction accumulation.
    """
    n = len(particles)
    total = 0.0

    for i in range(n):
        xi, yi = particles[i]
        for j in range(i + 1, n):
            xj, yj = particles[j]
            dx = xi - xj
            dy = yi - yj
            dist_sq = dx * dx + dy * dy + 1e-12  # avoid zero
            total += 1.0 / dist_sq

    return total


# -----------------------------
# Benchmark + profiling wrapper
# -----------------------------

def run_benchmark(n):
    particles = generate_particles(n)

    start = time.perf_counter()
    total = pairwise_interactions(particles)
    elapsed = time.perf_counter() - start

    return elapsed, total


# -----------------------------
# Main execution
# -----------------------------

if __name__ == "__main__":
    NUM_PARTICLES = 5_000  # O(N^2) — adjust carefully

    print(f"Number of particles: {NUM_PARTICLES}")

    tracemalloc.start()

    profiler = cProfile.Profile()
    profiler.enable()

    elapsed, total = run_benchmark(NUM_PARTICLES)

    profiler.disable()

    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    # ---- Results ----
    print("\nTiming Results (serial, pairwise interactions)")
    print(f"Elapsed time: {elapsed:.6f} s")
    print(f"Interaction total: {total:.6e}")

    print("\nMemory Usage")
    print(f"Current memory: {current / 1e6:.2f} MB")
    print(f"Peak memory:    {peak / 1e6:.2f} MB")

    # ---- Profiling output ----
    s = io.StringIO()
    stats = pstats.Stats(profiler, stream=s)
    stats.strip_dirs().sort_stats("cumulative").print_stats(10)

    print("\nTop 10 functions by cumulative time:")
    print(s.getvalue())


Number of particles: 5000

Timing Results (serial, pairwise interactions)
Elapsed time: 24.568911 s
Interaction total: 6.147630e+08

Memory Usage
Current memory: 0.16 MB
Peak memory:    0.53 MB

Top 10 functions by cumulative time:
         11120 function calls (11096 primitive calls) in 24.588 seconds

   Ordered by: cumulative time
   List reduced from 173 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        7    0.000    0.000   24.583    3.512 base_events.py:1922(_run_once)
      2/1    0.227    0.114   24.342   24.342 ipython-input-2377446676.py:41(run_benchmark)
       23   23.124    1.005   23.124    1.005 {built-in method time.sleep}
        1    0.991    0.991    0.991    0.991 ipython-input-2377446676.py:17(pairwise_interactions)
      8/5    0.000    0.000    0.034    0.007 events.py:86(_run)
      8/5    0.000    0.000    0.034    0.007 {method 'run' of '_contextvars.Context' objects}
        4    0.000    0.000    0